# Partial Reconstruction of a Tucker Tensor
```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

## Benefits of Partial Reconstruction
An advantage of Tucker decomposition is that the tensor can be partially reconstructed without ever forming the *full* tensor. The `reconstruct()` member function does this, resulting in significant time and memory savings, as we demonstrate below.

In [1]:
import pyttb as ttb
import numpy as np

## TODO: Decide if we want to use Miranda density tensor dataset
If so, the Miranda data is available at https://gitlab.com/tensors/tensor_data_miranda_sim. It loads a tensor `X` of size $384 \times 384 \times 256$.

## For now, we will use random data.

In [2]:
shape = (20, 30, 50)
X = ttb.tenrand(shape)

## Compute HOSVD
We compute the Tucker decomposition using ST-HOSVD with target relative error `0.001`.

In [3]:
%%time
T = ttb.hosvd(X, tol=0.001)

Computing HOSVD...

 Size of core: (20, 30, 50)
||X-T||/||X|| =  6.1027e-15 <= 0.001000 (tol)
CPU times: user 29.6 ms, sys: 63.5 ms, total: 93.1 ms
Wall time: 9.54 ms


In [4]:
# Note: If the result is < 1.0 x, it will be unsurprising 
# since the random generation process below wasn't expected 
# to return a low-rank approximation
print(
    f"Compression: {X.data.nbytes/(T.core.data.nbytes + np.sum([i.nbytes for i in T.factor_matrices]))} x"
)

Compression: 0.8875739644970414 x


## Full reconstruction
We can create a full reconstruction of the data using the full command. Not only is this expensive in computational time but also in memory. Now, let's see how long it takes to reconstruct the approximation to `X`.

In [5]:
%%time
Xf = T.full()

CPU times: user 9.22 ms, sys: 16.9 ms, total: 26.1 ms
Wall time: 1.93 ms


## Partial reconstruction
If we really only want part of the tensor, we can reconstruct just that part. Suppose we only want the `[:,15,:]` slice. The reconstruct function can do this much more efficiently with no loss in accuracy.

In [6]:
%%time
Xslice = T.reconstruct(modes=[1],samples=[15])

CPU times: user 6.28 ms, sys: 7.37 ms, total: 13.7 ms
Wall time: 1.88 ms


In [7]:
print(
    f"Compression: {Xf.data.nbytes/Xslice.data.nbytes} x"
)

Compression: 30.0 x


## Down-sampling
Additionally, we may want to downsample high-dimensional data to something lower resolution. For example, here we downsample in modes 1 and 2 by a factor of 2 and see even further speed-up and memory savings. There is no loss of accuracy as compared to downsampling after constructing the full tensor.

In [8]:
S0 = np.kron(np.eye(int(shape[0]/2)), 0.5*np.ones((1,2)))
S2 = np.kron(np.eye(int(shape[2]/2)), 0.5*np.ones((1,2)))
S1 = np.array([15])

In [9]:
%%time
Xds = T.reconstruct(modes=[0,1,2], samples=[S0,S1,S2])

CPU times: user 6.18 ms, sys: 8.22 ms, total: 14.4 ms
Wall time: 1.78 ms


In [10]:
print(
    f"Compression: {Xf.data.nbytes/Xds.data.nbytes} x"
)

Compression: 120.0 x


## TODO: Compare visualizations
We can compare the results of reconstruction. There is no degredation in doing only a partial reconstruction. Downsampling is obviously lower resolution, but the same result as first doing the full reconstruction and then downsampling.